<a href="https://colab.research.google.com/github/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/blob/main/algoritmo_ACND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from google.colab import files
import io
import numpy.ma as ma
from scipy.spatial import distance as dist

from sklearn.neighbors import KDTree
import statistics as stats

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
#Aggregation
dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/Aggregation.csv", encoding="ISO-8859-1" )
#Compound
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/Compound.csv", encoding="ISO-8859-1" )
#D31
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/D31.csv", encoding="ISO-8859-1" )
#Flame
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/Flame.csv", encoding="ISO-8859-1" )
#Jain
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/Jain.csv", encoding="ISO-8859-1" )
#Pathbased
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/Pathbased.csv", encoding="ISO-8859-1" )
#R15
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/R15.csv", encoding="ISO-8859-1" )
#Spiral
# dataset = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/Spiral.csv", encoding="ISO-8859-1" )

In [3]:
#Normalização

# dataset['d1'] = (dataset['d1'] - min(dataset['d1']))/(max(dataset['d1']) - min(dataset['d1']))
# dataset['d2'] = (dataset['d2'] - min(dataset['d2']))/(max(dataset['d2']) - min(dataset['d2']))

In [4]:
# Dataset com dados do artigo: (Sobre a etapa 5)
testando = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/datasets/teste.csv", encoding="ISO-8859-1" )

# Etapa 1: Read and analyze data, and determine parameters

In [5]:
def etapa1(data):
  N = data.shape[0]
  alfa = 0.0
  alpha = 0.0
  beta = 0.0
  gama = 0.0
  # print(data[0])
  for m in data:
    print(min(data[m]))
    max_s = max(data[m])
    min_s = min(data[m])
    rs = max_s - min_s
    alfa += rs

  alpha = N/alfa
  beta = ((3.5 * alpha) + 45) / 100
  gama = ( (2 * alpha) + 18)/100
  k = max(10, int(np.ceil((N * 0.01))))
  
  return [N,alfa,alpha,beta,gama,k]

In [6]:
etapa1(dataset)

3.35
1.95
1


[788,
 66.39999999999999,
 11.867469879518074,
 0.8653614457831327,
 0.4173493975903615,
 10]

#Etapa 2: Construct k-D Tree: kDT

In [7]:
def etapa2(data):
  kdt = KDTree(data)
  return kdt

#Etapa 3: Searches kNN of sample, calculates mean of 5NN’s

In [8]:
def etapa3(data,N,kdt):
  distance = []
  knn_index = []
  d5nn = []

  distance, knn_index = kdt.query(data,N)

  for i in range(len(knn_index)):
    d5nn.append(np.mean(distance[i, 1:6]))

  print("KNN:")
  for i in range(N):
    print(distance[i][1],knn_index[i][1:16])

  return [distance, knn_index, d5nn]

#Etapa 4: Calculate the global radius of SR

In [9]:
def etapa4(d5nn):
  d5nn = np.sort(d5nn)
  R = max(d5nn)
  print("R:",R)
  return [d5nn, R]

#Etapa 5: Calculate sample density array(Rho)

In [10]:
def fab(i,j,a,b,distance):
  d = distance[i,j]
  if d >= 0 and d<= a:
    return 1
  elif d > a and d <= b:
    return ((b-d)/ (b-a))
  elif d > b:
    return 0

In [11]:
def etapa5(N,R,distance):
  surrounding_region = []
  Rho = []
  for i in range(N):
    Rho.append(0)

  for i in range(N):
    surrounding_region = distance[i,] < R
    surrounding_region = distance[i,surrounding_region]
    surrounding_mean = np.mean(surrounding_region)
    surrounding_std_dev = stats.stdev(surrounding_region)
    surrounding_region = np.sort(surrounding_region)

    nc = len(surrounding_region)

    d1 = np.mean(surrounding_region[:(np.int(np.ceil(nc*0.1))+1)])
    d2 = min(surrounding_region[1:])

    a = (d1+d2)/2
    b = min((a+(2*surrounding_std_dev)),R)

    # a = testando.a[i]
    # b = testando.b[i]

    for j in range(0,len(surrounding_region)):
      f = fab(i,j,a,b, distance)
      Rho[i] += f

    print(i, "nc:",nc, " a: %.5f"  %a, " b: %.5f"%b, " Rho: %.5f" %Rho[i])
  return [surrounding_region, Rho]

#Etapa 6: Calculate the global density threshold(T)

In [12]:
def etapa6(N,rho):
  arr = []
  for i in range(N):
    arr.append((rho[i], i))

  arr.sort()
  arr_invert = arr[::-1]

  df_arr = pd.DataFrame(arr_invert)
  t = np.percentile(df_arr[0], 25)

  print("T:",t)
  return [t,arr_invert]

#Etapa 7: Clustering

In [13]:
def existe(arr, valor):
  for i in arr:
    if i == valor:
      return True
  return False

In [14]:
# Feito 2 queue para registrar que já passou por determinada amostra, se não, entra em loop infinito
def etapa7(N,R,arr,t,rho,k,knn_index,distance,beta,gama ):
  order_of_cluster = 0
  tag = []
  for i in range(N):
    tag.append(False)
  queue = []
  queue_2 = []

  result = []
  for y in range(N):
    result.append(0)

  for ind in range(N):
    # 1 representa indice e 0 valor de arr
    if ( tag[np.int(arr[ind][1])]):
      continue
    if (arr[ind][0] == 0):
      tag[np.int(arr[ind][1])] = True
      continue
    if ( arr[ind][0] < t ):
      continue

    order_of_cluster += 1
    tag[ np.int(arr[ind][1]) ] = True
    queue.clear()
    queue_2.clear()
    queue.append(arr[ind][1])
    queue_2.append(arr[ind][1])

    while (len(queue) > 0):
      i = queue.pop(0)

      aver = (R + rho[np.int(i)]) / 2
      local_t = aver * beta
      # local_t = 0.806623
      for e in range(k):
        j = knn_index[np.int(i)][e]

        if (distance[np.int(i)][e] > R):
          continue
        
        co_nn = 0
        for a in range(1,k):
          for b in range(1,k):
            if (knn_index[np.int(i)][a] == knn_index[j][b]):
              co_nn += 1

        tag[j] = True

        if (rho[j]):
          result[j] = order_of_cluster 
          if (rho[j] >= local_t and co_nn > (k * gama)):   #(Rho[j] > = localT && co_NN > k ∗γ )

            if (not existe(queue_2,j)):
              queue.append(j)
              queue_2.append(j)
              # print(j)

  # print("Resultado parcial:",result)

  return [result,order_of_cluster]

#Etapa 8: Post–Process clustering and save the results

In [15]:
def etapa8(N,order_of_cluster,result):

  for cluster in range(1,order_of_cluster):
    cont = 0
    for i in result:
      if (i == cluster):
        cont += 1

    if ( cont <= 5 ):
      for j in range(N):
        if (result[j] == cluster ):
          result[j] = 0

  resultLenToPrint = np.int(np.ceil((len(result)/10)))
  for i in range(resultLenToPrint):
    print(i*10,end=" ")
    if i == (resultLenToPrint-1):
      for j in range(len(result)%10):
        print(result[( (i*10) + j )],end=" ")
    else:
      for j in range(10):
        print(result[( (i*10) + j )],end=" ")
    print(" ")

  return result

Plot dos gráficos

In [16]:
from google.colab import files

In [17]:
def plotar(data, out,name):
  g =sns.scatterplot(x="d1", y="d2",
                hue=out,palette ="Paired",
                data=data)
  plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0)
  plt.savefig(name+".png")
  files.download(name+".png") 
  plt.show()

In [18]:
def plotarGraficos(data_in, out_acnd, out_origin, name):
  plotar(data_in,out_origin, name+"_correct")
  plotar(data_in,out_acnd, name+"_acnd")

# Função do algoritmo:

In [19]:
def acnd(dataset, data_name):
  data = dataset[['d1','d2']]
  saida = dataset['cluster']
  N,alfa,alpha,beta,gama,k = etapa1(data)
  # beta = 0.35
  # gama = 0.4
  print([N,alfa,alpha,beta,gama,k])
  kdt = etapa2(data)
  # print(kdt)

  distance,knn_index,d5nn = etapa3(data, N, kdt)
  # print(distance)

  d5nn, R = etapa4(d5nn)
  # print(R)

  surrounding_region, Rho = etapa5(N,R,distance)
  # print(rho)
  # Rho = testando.fv

  T,Arr = etapa6(N,Rho)
  # print(T)
  # T = 4.11459
  result, order_clusters = etapa7(N,R,Arr,T,Rho,k,knn_index,distance,beta,gama)

  result = etapa8(N,order_clusters,result)

  plotarGraficos(data,result,saida,data_name) 

In [ ]:
from sklearn.datasets import make_blobs
n_features = 2
n_samples = 200
n_clusters = 5

dados_blobs = make_blobs(n_samples=n_samples, centers=n_clusters, n_features=n_features,random_state=0)

dataset_1 = {}

dataset_1['d1'] = []
dataset_1['d2'] = []
dataset_1['cluster'] = []
for i in range(n_samples):
  dataset_1['d1'].append(dados_blobs[0][i][0])
  dataset_1['d2'].append(dados_blobs[0][i][1])
  dataset_1['cluster'].append(dados_blobs[1][i])
print(dataset_1)

df = pd.DataFrame.from_dict(dataset_1)

In [21]:
# acnd(df)

In [ ]:
acnd(dataset,"Aggregation")

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
from sklearn.cluster import KMeans

clusterss_number = len(dataset['cluster'].unique())

clustering = DBSCAN(eps=2,min_samples=2).fit_predict(dataset)
brc = Birch(n_clusters=clusterss_number,threshold=0.5)
brc.fit(dataset)
y_brc = brc.predict(dataset)
KMeans_model = KMeans(n_clusters=clusterss_number,random_state=0).fit(dataset)
y_kmeans = KMeans_model.predict(dataset)


data = dataset[['d1','d2']]

nome = "" #Colocar nome do dataset que estiver utilizando

plotar(data,clustering,nome+ "_DBSCAN")
plotar(data,y_brc,nome+ "_Birch")
plotar(data,y_kmeans,nome+ "_KMeans")